# Step 1 - Double Check proper enviroment setup

* write this in terminal "conda activate pii
python -m ipykernel install --user --name=pii --display-name "Python (pii)"

In [86]:
import sys
print(sys.executable)

/opt/miniconda3/envs/pii/bin/python


In [87]:
# Quick check that key packages are available
import datasets
import transformers
import torch
import pandas

print(f"datasets version: {datasets.__version__}")
print(f"transformers version: {transformers.__version__}")
print(f"torch version: {torch.__version__}")
print(f"torch version: {pandas.__version__}")

datasets version: 4.4.1
transformers version: 4.57.1
torch version: 2.9.1
torch version: 2.3.3


In [88]:
from datasets import get_dataset_config_names, get_dataset_split_names
from huggingface_hub import list_repo_files

# See what files are actually in the repository
files = list_repo_files("tursunait/deberta-pii-synth", repo_type="dataset")
print("Files in the repository:")
for f in files:
    print(f"  {f}")

Files in the repository:
  .gitattributes
  README.md
  test/data-00000-of-00001.arrow
  test/dataset_info.json
  test/state.json
  train/.gitattributes
  train/README.md
  train/data-00000-of-00002.arrow
  train/data-00001-of-00002.arrow
  train/dataset_info.json
  train/state.json
  val/data-00000-of-00001.arrow
  val/dataset_info.json
  val/state.json


# Step 2: Load and Analyse Dataset

* Issue: This confirms the issue: train uses Arrow format, but validation and test use JSON format! This is a misconfiguration in the dataset on Hugging Face itself.The dataset IS actually all in Arrow format, but there's a configuration issue preventing it from loading automatically. So use the exact code below not the one given by Tursunai.

In [89]:
from datasets import load_dataset

# Load by explicitly pointing to the arrow files
ds = load_dataset(
    "arrow",
    data_files={
        "train": "hf://datasets/tursunait/deberta-pii-synth/train/data-*.arrow",
        "validation": "hf://datasets/tursunait/deberta-pii-synth/val/data-*.arrow",
        "test": "hf://datasets/tursunait/deberta-pii-synth/test/data-*.arrow"
    }
)

# Dataset already split
train = ds["train"]
val = ds["validation"]
test = ds["test"]

print("Train size:", len(train))
print("Val size:", len(val))
print("Test size:", len(test))
print("\nFirst example:")
print(train[0])

Train size: 96000
Val size: 12000
Test size: 12000

First example:
{'text': 'Jack Phillips fr7m Hancock-Melendez uqes card 4843127370283685 on 1994-08-27.', 'spans': [{'end': 13, 'label': 'PERSON', 'start': 0}, {'end': 35, 'label': 'ORG', 'start': 19}, {'end': 62, 'label': 'CREDIT_CARD', 'start': 46}, {'end': 76, 'label': 'DATE', 'start': 66}], 'input_ids': [1, 20907, 7431, 6664, 406, 119, 19632, 12, 20201, 22192, 1717, 1343, 293, 1886, 2929, 3897, 1092, 5352, 3083, 2517, 3367, 4531, 15, 8148, 12, 3669, 12, 2518, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
import numpy as np
print("🔍 Analyzing FULL training dataset...")

# Check ALL training examples
lengths = [len(example['input_ids']) for example in train]

print(f"\n📊 Sequence Length Statistics:")
print(f"Total examples: {len(train):,}")
print(f"Average length: {np.mean(lengths):.1f} tokens")
print(f"Median length: {np.median(lengths):.1f} tokens")
print(f"Min length: {np.min(lengths)} tokens")
print(f"Max length: {np.max(lengths)} tokens")
print(f"Std deviation: {np.std(lengths):.1f}")

# Distribution analysis
print(f"\n📈 Length Distribution:")
print(f"Sequences < 50 tokens: {sum(1 for l in lengths if l < 50):,} ({sum(1 for l in lengths if l < 50)/len(lengths)*100:.1f}%)")
print(f"Sequences 50-100 tokens: {sum(1 for l in lengths if 50 <= l < 100):,} ({sum(1 for l in lengths if 50 <= l < 100)/len(lengths)*100:.1f}%)")
print(f"Sequences 100-200 tokens: {sum(1 for l in lengths if 100 <= l < 200):,} ({sum(1 for l in lengths if 100 <= l < 200)/len(lengths)*100:.1f}%)")
print(f"Sequences 200-512 tokens: {sum(1 for l in lengths if 200 <= l < 512):,} ({sum(1 for l in lengths if 200 <= l < 512)/len(lengths)*100:.1f}%)")
print(f"Sequences 512+ tokens: {sum(1 for l in lengths if l >= 512):,} ({sum(1 for l in lengths if l >= 512)/len(lengths)*100:.1f}%)")

print("\n⏱️ Time Estimation:")
print(f"With batch_size=8: ~{len(train)//8:,} steps per epoch")
print(f"Estimated time at ~0.5 sec/step: {(len(train)//8 * 0.5)/3600:.1f} hours per epoch")

🔍 Analyzing FULL training dataset...

📊 Sequence Length Statistics:
Total examples: 96,000
Average length: 256.0 tokens
Median length: 256.0 tokens
Min length: 256 tokens
Max length: 256 tokens
Std deviation: 0.0

📈 Length Distribution:
Sequences < 50 tokens: 0 (0.0%)
Sequences 50-100 tokens: 0 (0.0%)
Sequences 100-200 tokens: 0 (0.0%)
Sequences 200-512 tokens: 96,000 (100.0%)
Sequences 512+ tokens: 0 (0.0%)

⏱️ Time Estimation:
With batch_size=8: ~12,000 steps per epoch
Estimated time at ~0.5 sec/step: 1.7 hours per epoch


# Step 3: Build Model

## 3.1 - Load DeBERTa-base
- DeBERTa is a pre-trained language model (like a smart AI that already understands language). You need to:

    - Load the base model called "microsoft/deberta-base"
    - Load it specifically for token classification (labeling each word/token)\
    - The model needs to know how many labels exist (like PERSON, ORG, EMAIL, etc.)
    - Also load the tokenizer (converts text to numbers the model understands)

- Think of it like: You're taking a smart student (DeBERTa) who already knows English, and now you're going to teach them to specifically identify PII in text.


## 3.2 - Baseline Model: Fixed Hyperparameter

### A. Train and Validation

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
import sys
import os

# Add parent directory to path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Import label configuration
from pii_synth.config_and_labels import LABEL2ID, ID2LABEL

# Step 1: Call the model - RoBERTa (Most stable and popular for NER)
model_name = "roberta-base"

num_labels = len(ID2LABEL)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=ID2LABEL,
    label2id=LABEL2ID
)

# Step 2: Create SMALL subsets for fast hyperparameter tuning - IF WE USE FULL MODEL IT BREAKS
train_subset = train.select(range(3000))
val_subset = val.select(range(500))

print(f"Train subset: {len(train_subset)} examples")
print(f"Val subset: {len(val_subset)} examples")

# Step 3: Training configuration - STABLE SETTINGS -- 
training_args = TrainingArguments(
    output_dir="./results_baseline_model",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="no",
    report_to="none",
    max_grad_norm=1.0,
    warmup_ratio=0.1,
    weight_decay=0.01,
    dataloader_num_workers=0,
)

# Step 4: Data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Step 5: Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=val_subset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Step 6: Train
print(f"\n Starting training with {model_name}...")
trainer.train()
print("✅ Training complete!")

# Step 7: Evaluate
print("\n📈 Evaluating...")
results = trainer.evaluate()
print(f"Validation results: {results}")

🔄 Loading roberta-base...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/_5/g20j99hd5j91r849_d_zgb040000gn/T/ipykernel_54221/1821411500.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


✅ Model loaded with 33 labels
📊 Creating small subsets for fast tuning...
Train subset: 3000 examples
Val subset: 500 examples

🚀 Starting training with roberta-base...
⏱️  This should work without overflow errors!


Step,Training Loss,Validation Loss


✅ Training complete!

📈 Evaluating...


Validation results: {'eval_loss': 0.02069513127207756, 'eval_runtime': 25.2524, 'eval_samples_per_second': 19.8, 'eval_steps_per_second': 2.495, 'epoch': 2.0}


### B. Test

In [113]:
import numpy as np

# (Optional) use smaller test subset
test_eval = test.select(range(300))   # or test for full set

# 1) Run predictions on the test set
test_predictions = trainer.predict(test_eval)

logits = test_predictions.predictions   # [batch, seq_len, num_labels]
labels = test_predictions.label_ids     # [batch, seq_len]

# 2) Convert logits → predicted label IDs
pred_ids = np.argmax(logits, axis=-1)

# 3) Remove padding tokens (label == -100)
all_preds = []
all_labels = []

for p, t in zip(pred_ids, labels):
    mask = t != -100
    all_preds.append(p[mask])
    all_labels.append(t[mask])

# Flatten for simple metrics
all_preds_flat = np.concatenate(all_preds)
all_labels_flat = np.concatenate(all_labels)

print("Test prediction step done.")


/opt/miniconda3/envs/pii/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Test prediction step done.


### C. Performance Evaluation - METRICS

In [114]:
from sklearn.metrics import accuracy_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

# 1) Token-level accuracy (mostly meaningless for NER but fine to report)
accuracy = accuracy_score(all_labels_flat, all_preds_flat)
print("Token-level Accuracy:", accuracy)

# 2) Convert label IDs → text labels for seqeval
id2label = ID2LABEL   # your mapping from earlier

grouped_preds = []
grouped_labels = []

for p_seq, t_seq in zip(all_preds, all_labels):
    grouped_preds.append([id2label[int(i)] for i in p_seq])
    grouped_labels.append([id2label[int(i)] for i in t_seq])

# 3) Print real NER metrics
print("\nPrecision:", precision_score(grouped_labels, grouped_preds))
print("Recall:", recall_score(grouped_labels, grouped_preds))
print("F1 Score:", f1_score(grouped_labels, grouped_preds))

print("\nDetailed classification report:")
print(classification_report(grouped_labels, grouped_preds))


Token-level Accuracy: 0.9978969505783386

Precision: 0.9803571428571428
Recall: 0.987410071942446
F1 Score: 0.9838709677419355

Detailed classification report:
              precision    recall  f1-score   support

     ADDRESS       0.98      1.00      0.99        44
 CREDIT_CARD       0.98      1.00      0.99        61
        DATE       1.00      1.00      1.00       107
       EMAIL       0.97      0.97      0.97        61
         ORG       0.99      0.99      0.99        96
      PERSON       0.96      0.97      0.96       112
       PHONE       1.00      1.00      1.00        57
         SSN       0.94      0.94      0.94        18

   micro avg       0.98      0.99      0.98       556
   macro avg       0.98      0.98      0.98       556
weighted avg       0.98      0.99      0.98       556



/opt/miniconda3/envs/pii/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/miniconda3/envs/pii/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-PHONE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/miniconda3/envs/pii/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-PERSON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/miniconda3/envs/pii/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/miniconda3/envs/pii/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-EMAIL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk